In [1]:
import youtube_dl
import os
import sys
import shutil
from moviepy.editor import VideoFileClip, concatenate_videoclips
import numpy as np

# from VideoSync.alignment_by_row_channels import *

from cvcalib.audiosync import offset

curr_dir = "/home/sean/Documents/experiments/stage-mix-generator/"
download_dir = os.path.join(curr_dir, 'temp')

In [13]:
# get videos via youtube api
lyrics_video_url = "https://www.youtube.com/watch?v=mIXR-7u2Kas"
stage_video_urls = [
    "https://www.youtube.com/watch?v=kda_53A0ZpI",
    "https://www.youtube.com/watch?v=zzq8uI1BnJk",
    "https://www.youtube.com/watch?v=Q65bS5EyX5I",
    "https://www.youtube.com/watch?v=547LCSYK8Xg",
    
]

In [14]:
# download videos

ydl_opts = {
    'format': 'mp4',
    'outtmpl': os.path.join(download_dir, '%(id)s.%(ext)s')
}
with youtube_dl.YoutubeDL(ydl_opts) as ydl:
    ydl.download([lyrics_video_url])
    meta = ydl.extract_info(
        lyrics_video_url, download=False)
    
    shutil.move(os.path.join(download_dir, f"{meta['id']}.mp4"), os.path.join(download_dir, "lyrics_video.mp4"))
    
    for i, url in enumerate(stage_video_urls):
        ydl.download([url])
        meta = ydl.extract_info(
            url, download=False)
        
        shutil.move(os.path.join(download_dir, f"{meta['id']}.mp4"), os.path.join(download_dir, f"stage_video{i}.mp4"))

[youtube] mIXR-7u2Kas: Downloading webpage
[youtube] mIXR-7u2Kas: Downloading video info webpage
[download] Destination: /home/sean/Documents/experiments/stage-mix-generator/temp/mIXR-7u2Kas.mp4
[download] 100% of 8.88MiB in 00:0087MiB/s ETA 00:001
[youtube] mIXR-7u2Kas: Downloading webpage
[youtube] mIXR-7u2Kas: Downloading video info webpage
[youtube] kda_53A0ZpI: Downloading webpage
[youtube] kda_53A0ZpI: Downloading video info webpage
[download] Destination: /home/sean/Documents/experiments/stage-mix-generator/temp/kda_53A0ZpI.mp4
[download] 100% of 25.29MiB in 00:0198MiB/s ETA 00:007
[youtube] kda_53A0ZpI: Downloading webpage
[youtube] kda_53A0ZpI: Downloading video info webpage
[youtube] zzq8uI1BnJk: Downloading webpage
[youtube] zzq8uI1BnJk: Downloading video info webpage
[download] Destination: /home/sean/Documents/experiments/stage-mix-generator/temp/zzq8uI1BnJk.mp4
[download] 100% of 21.99MiB in 00:0005MiB/s ETA 00:003
[youtube] zzq8uI1BnJk: Downloading webpage
[youtube] zzq8

In [15]:
lyrics_video_path = os.path.join(download_dir, 'lyrics_video.mp4')
stage_video_paths = [ os.path.join(download_dir, f'stage_video{i}.mp4') for i in range(len(stage_video_urls)) ]

# find timestamp of start of song in stage videos
# t = align('lyrics_video.mp4', 'stage_video1.mp4', download_dir+'/')

offset.find_time_offset(['lyrics_video.mp4', 'stage_video0.mp4'], download_dir+'/', [0, 0])

((0, 32.8446), 29.97)

In [16]:
stage_song_starts = []
for i in range(len(stage_video_urls)):
    output = offset.find_time_offset(['lyrics_video.mp4', f'stage_video{i}.mp4'], download_dir+'/', [0, 0])
    print(output)
    stage_song_starts.append(output[0][1])

((0, 32.8446), 29.97)
((0, 0.1161), 29.97)
((0, 7.0937), 29.97)
((0, 15.6851), 29.97)


In [17]:
# get subclips of the song in the stage videos

# get length of lyrics video
song = VideoFileClip(lyrics_video_path)

stage_videos = []
for i, path in enumerate(stage_video_paths):
    stage_videos.append(VideoFileClip(path).subclip(stage_song_starts[i]))

In [19]:
# every 5 seconds during the song, splice clips from different performances

clips = []
stage_video_idx = 0
for step in np.arange(0, song.duration-2, 5):
    clips.append(stage_videos[stage_video_idx].subclip(step, step+5))
    
    stage_video_idx = (stage_video_idx + 1) % len(stage_videos)
    
final_clip = concatenate_videoclips(clips)
final_clip.write_videofile("stage_mix.mp4")

                                                                     
chunk:  99%|█████████▊| 4783/4852 [07:57<00:00, 521.41it/s, now=None]
                                                                     
chunk:  99%|█████████▉| 4805/4852 [00:43<00:00, 669.26it/s, now=None]

chunk:   0%|          | 0/4741 [00:00<?, ?it/s, now=None]

chunk:   0%|          | 2/4741 [00:00<04:23, 17.99it/s, now=None]

Moviepy - Building video stage_mix.mp4.
MoviePy - Writing audio in %s




chunk:   1%|          | 32/4741 [00:00<03:08, 25.04it/s, now=None]

chunk:   1%|▏         | 67/4741 [00:00<02:14, 34.70it/s, now=None]

chunk:   2%|▏         | 112/4741 [00:00<01:39, 46.72it/s, now=None]

chunk:   4%|▎         | 173/4741 [00:00<01:10, 64.60it/s, now=None]

chunk:   5%|▍         | 222/4741 [00:00<00:54, 83.59it/s, now=None]

chunk:   6%|▌         | 296/4741 [00:00<00:39, 113.90it/s, now=None]

chunk:   7%|▋         | 340/4741 [00:01<00:33, 133.10it/s, now=None]

chunk:   9%|▊         | 407/4741 [00:01<00:24, 174.85it/s, now=None]

chunk:  10%|▉         | 453/4741 [00:01<00:20, 213.74it/s, now=None]

chunk:  11%|█▏        | 540/4741 [00:01<00:15, 276.07it/s, now=None]

chunk:  13%|█▎        | 603/4741 [00:01<00:12, 331.30it/s, now=None]

chunk:  14%|█▍        | 670/4741 [00:01<00:10, 390.18it/s, now=None]

chunk:  16%|█▌        | 762/4741 [00:01<00:08, 471.22it/s, now=None]

chunk:  18%|█▊        | 834/4741 [00:01<00:07, 525.35it/s, now=None]

chunk:  19%|█▉        | 9

MoviePy - Done.
Moviepy - Writing video stage_mix.mp4





t:   1%|          | 49/6444 [00:00<01:34, 67.38it/s, now=None]

t:   1%|          | 63/6444 [00:00<01:20, 79.61it/s, now=None]

t:   1%|▏         | 84/6444 [00:00<01:07, 94.82it/s, now=None]

t:   2%|▏         | 105/6444 [00:00<00:56, 112.97it/s, now=None]

t:   2%|▏         | 122/6444 [00:00<00:54, 115.55it/s, now=None]

t:   2%|▏         | 138/6444 [00:00<00:58, 107.14it/s, now=None]

t:   2%|▏         | 152/6444 [00:01<01:12, 87.31it/s, now=None] 

t:   3%|▎         | 181/6444 [00:01<00:56, 110.26it/s, now=None]

t:   3%|▎         | 198/6444 [00:01<00:52, 119.50it/s, now=None]

t:   3%|▎         | 214/6444 [00:01<00:51, 119.85it/s, now=None]

t:   4%|▎         | 229/6444 [00:01<00:49, 124.98it/s, now=None]

t:   4%|▍         | 244/6444 [00:01<00:51, 120.80it/s, now=None]

t:   4%|▍         | 259/6444 [00:01<00:49, 126.10it/s, now=None]

t:   4%|▍         | 274/6444 [00:01<00:48, 127.90it/s, now=None]

t:   4%|▍         | 289/6444 [00:01<00:46, 133.57it/s, now=None]

t:   5%|▍     

t:  28%|██▊       | 1832/6444 [00:14<00:34, 133.18it/s, now=None]

t:  29%|██▊       | 1849/6444 [00:14<00:32, 139.60it/s, now=None]

t:  29%|██▉       | 1866/6444 [00:14<00:32, 138.86it/s, now=None]

t:  29%|██▉       | 1882/6444 [00:14<00:32, 141.24it/s, now=None]

t:  29%|██▉       | 1898/6444 [00:14<00:34, 131.93it/s, now=None]

t:  30%|██▉       | 1914/6444 [00:14<00:33, 134.53it/s, now=None]

t:  30%|██▉       | 1929/6444 [00:14<00:35, 128.61it/s, now=None]

t:  30%|███       | 1943/6444 [00:15<00:35, 127.70it/s, now=None]

t:  30%|███       | 1957/6444 [00:15<00:46, 97.12it/s, now=None] 

t:  31%|███       | 1978/6444 [00:15<00:38, 114.84it/s, now=None]

t:  31%|███       | 1992/6444 [00:15<00:38, 117.16it/s, now=None]

t:  31%|███       | 2007/6444 [00:15<00:35, 123.29it/s, now=None]

t:  31%|███▏      | 2021/6444 [00:15<00:35, 123.58it/s, now=None]

t:  32%|███▏      | 2035/6444 [00:15<00:36, 122.05it/s, now=None]

t:  32%|███▏      | 2050/6444 [00:15<00:34, 127.49it/s, now=No

t:  57%|█████▋    | 3642/6444 [00:27<00:20, 137.45it/s, now=None]

t:  57%|█████▋    | 3657/6444 [00:27<00:20, 135.47it/s, now=None]

t:  57%|█████▋    | 3673/6444 [00:27<00:19, 141.11it/s, now=None]

t:  57%|█████▋    | 3688/6444 [00:28<00:20, 136.06it/s, now=None]

t:  57%|█████▋    | 3702/6444 [00:28<00:20, 134.63it/s, now=None]

t:  58%|█████▊    | 3716/6444 [00:28<00:20, 133.25it/s, now=None]

t:  58%|█████▊    | 3733/6444 [00:28<00:19, 142.06it/s, now=None]

t:  58%|█████▊    | 3749/6444 [00:28<00:26, 99.91it/s, now=None] 

t:  59%|█████▊    | 3775/6444 [00:28<00:21, 122.34it/s, now=None]

t:  59%|█████▉    | 3792/6444 [00:28<00:21, 124.45it/s, now=None]

t:  59%|█████▉    | 3808/6444 [00:29<00:21, 123.73it/s, now=None]

t:  59%|█████▉    | 3823/6444 [00:29<00:20, 127.39it/s, now=None]

t:  60%|█████▉    | 3838/6444 [00:29<00:20, 127.82it/s, now=None]

t:  60%|█████▉    | 3852/6444 [00:29<00:20, 124.20it/s, now=None]

t:  60%|██████    | 3868/6444 [00:29<00:19, 131.50it/s, now=No

t:  84%|████████▍ | 5439/6444 [00:41<00:07, 130.79it/s, now=None]

t:  85%|████████▍ | 5454/6444 [00:41<00:07, 135.90it/s, now=None]

t:  85%|████████▍ | 5470/6444 [00:41<00:06, 142.16it/s, now=None]

t:  85%|████████▌ | 5485/6444 [00:41<00:06, 141.17it/s, now=None]

t:  85%|████████▌ | 5500/6444 [00:41<00:06, 137.20it/s, now=None]

t:  86%|████████▌ | 5516/6444 [00:42<00:06, 141.75it/s, now=None]

t:  86%|████████▌ | 5531/6444 [00:42<00:06, 141.64it/s, now=None]

t:  86%|████████▌ | 5546/6444 [00:42<00:06, 138.41it/s, now=None]

t:  86%|████████▋ | 5560/6444 [00:42<00:07, 113.05it/s, now=None]

t:  86%|████████▋ | 5574/6444 [00:42<00:07, 119.80it/s, now=None]

t:  87%|████████▋ | 5587/6444 [00:42<00:07, 118.59it/s, now=None]

t:  87%|████████▋ | 5600/6444 [00:42<00:07, 116.95it/s, now=None]

t:  87%|████████▋ | 5614/6444 [00:42<00:06, 122.41it/s, now=None]

t:  87%|████████▋ | 5627/6444 [00:42<00:06, 123.76it/s, now=None]

t:  88%|████████▊ | 5640/6444 [00:43<00:06, 115.40it/s, now=No

Moviepy - Done !
Moviepy - video ready stage_mix.mp4
